In [1]:
import os
import time
import pandas as pd
import numpy as np
import pathlib
from io import StringIO
from Bio import SeqIO
from Bio.Cluster import distancematrix
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import subprocess
from matplotlib import pyplot as plt

In [27]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

MagicBlast:

- raw reads to nt
- contigs to nt
- raw reads to gsa_virus
- contigs to gsa_virus

In [2]:
PRJ='PRJCA002202'
DATA_PATH=f'/mnt/1TB_0/Data/Assembly/{PRJ}/'


In [3]:
def clean_string(s):
    s = s.replace(',', '').strip()
    return s

In [4]:
def get_contigs_ascessions(contigs_file):
    contigs=[]
    accessions=[]
    with open(contigs_file, 'r') as temp_f:
        for line in temp_f:
            if len(line.strip())>0:
                vals=line.split("\t")
                contigs.append(clean_string(vals[0]))
                accessions.append(clean_string(vals[2]))
    return  contigs, accessions


In [5]:
def get_raw_sam_ascessions(reads_file, machine_id='v300043428'):
    accessions=[]
    with open(reads_file, 'r') as temp_f:
        for line in temp_f:
            if len(line.strip())>0:
                if line.startswith(machine_id):
                    vals=line.split("\t")
                    accessions.append(clean_string(vals[2]))
    return accessions

In [6]:
def get_val_count(accessions):
    values, counts = np.unique(accessions, return_counts=True)
    idx = np.argsort(counts)[::-1]
    values = np.array(values)[idx]
    counts = np.array(counts)[idx]
    return values, counts, idx

In [7]:
def get_titles(df, values):
    titles=[]
    for v in values:
        l=df.loc[df.accession==v, 'title'].tolist()
        titles.append(l[0])
    return titles

In [8]:
def get_accession_dat(values, dbname='nt'):
    vdats=[]
    for v in values:
        try:
            vdat=!blastdbcmd -db nt -entry $v
        except Exception as e:
            if dbname!='nt':
                vdat=!blastdbcmd -db 'nt' -entry $v
        if vdat is not None:
            vdats.append(vdat[0].split(',')[0])
    return vdats

## Raw Reads

### All CRR

In [9]:
sra_list=['CRR125934','CRR125935','CRR125936','CRR125937','CRR125938','CRR125939','CRR125940']

In [12]:
sra_results={}
for sra in sra_list:
    print(f'SRA: {sra}')
    reads_file=f'{sra}_raw_magicBLAST_gsa_bsl_nt_db.sam'
    reads_file=DATA_PATH+f'{sra}/magic_blast/'+reads_file
    accessions=get_raw_sam_ascessions(reads_file, machine_id='M04403')
    values, counts, idx = get_val_count(accessions)
    titles=get_accession_dat(values, dbname='gsa_bsl_nt_db')
    for t, v,c in zip(titles, values, counts):
        print(f'{t}, {v}, {c}')
    sra_results[sra]=[titles, values, counts]
        

SRA: CRR125934
Error: [blastdbcmd] Entry not found: CNA0013710, CNA0013710, 72786
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 5816
>MT241668.1 Leopoldamys sabanus voucher MZF1958 mitochondrion, MT241668.1, 4334
>X51700.1 Bos taurus mRNA for bone Gla protein, 7, 3028
>MK280367.1 Homo sapiens lncAB371.6 lncRNA gene, MK280367.1, 890
>MK279923.1 Homo sapiens lncAB599.3 lncRNA gene, MK279923.1, 814
Error: [blastdbcmd] Entry not found: 2, 2, 744
>AJ289709.1 Human endogenous retrovirus H HERV-H/env62 proviral copy, AJ289709.1, 652
Error: [blastdbcmd] Entry not found: MW635910.1, MW635910.1, 618
>MW206626.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-1805/2020 ORF1ab polyprotein (ORF1ab) and ORF1a polyprotein (ORF1ab) genes, MW206626.1, 618
Error: [blastdbcmd] Entry not found: MW637224.1, MW637224.1, 618
>AC123074.2 Mus musculus BAC clone RP23-168H19 from 10, AC123074.2, 556
>MT520543.1 Severe acute respiratory syndrome coronav

Error: [blastdbcmd] Entry not found: MW505431.1, MW505431.1, 20
Error: [blastdbcmd] Entry not found: MW643347.1, MW643347.1, 20
Error: [blastdbcmd] Entry not found: MW505814.1, MW505814.1, 20
Error: [blastdbcmd] Entry not found: MW668901.1, MW668901.1, 20
>MW206526.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-1856/2020 ORF1ab polyprotein (ORF1ab) and ORF1a polyprotein (ORF1ab) genes, MW206526.1, 20
Error: [blastdbcmd] Entry not found: MW626204.1, MW626204.1, 20
>LR824567.1 Severe acute respiratory syndrome coronavirus 2 isolate 140089 genome assembly, LR861099.1, 20
Error: [blastdbcmd] Entry not found: MW495854.1, MW495854.1, 20
>LR824135.1 Severe acute respiratory syndrome coronavirus 2 isolate 120011 genome assembly, LR860690.1, 20
Error: [blastdbcmd] Entry not found: MW495137.1, MW495137.1, 20
>MW184296.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/AUS/VIC15041/2020 ORF1ab polyprotein (ORF1ab) and ORF1a polyprote

Error: [blastdbcmd] Entry not found: MW640818.1, MW640818.1, 12
Error: [blastdbcmd] Entry not found: MW637442.1, MW637442.1, 12
Error: [blastdbcmd] Entry not found: MW556180.1, MW556180.1, 12
Error: [blastdbcmd] Entry not found: MW637457.1, MW637457.1, 12
Error: [blastdbcmd] Entry not found: MW637458.1, MW637458.1, 12
Error: [blastdbcmd] Entry not found: MW637477.1, MW637477.1, 12
Error: [blastdbcmd] Entry not found: MW555917.1, MW555917.1, 12
Error: [blastdbcmd] Entry not found: MW637260.1, MW637260.1, 12
Error: [blastdbcmd] Entry not found: MW634865.1, MW634865.1, 12
Error: [blastdbcmd] Entry not found: MW566839.1, MW566839.1, 12
Error: [blastdbcmd] Entry not found: MW564995.1, MW564995.1, 12
Error: [blastdbcmd] Entry not found: MW565821.1, MW565821.1, 12
>MW449465.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/UT-UPHL-2012959648/2020 ORF1ab polyprotein (ORF1ab) gene, MW449465.1, 12
Error: [blastdbcmd] Entry not found: MW639514.1, MW639514.1, 12
Error:

Error: [blastdbcmd] Entry not found: MW513651.1, MW513651.1, 8
Error: [blastdbcmd] Entry not found: MW513646.1, MW513646.1, 8
Error: [blastdbcmd] Entry not found: MW555320.1, MW555320.1, 8
>MT834670.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-0512/2020 ORF1ab polyprotein (ORF1ab), MT834670.1, 8
>MT834669.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-0511/2020 ORF1ab polyprotein (ORF1ab), MT834669.1, 8
>MT834662.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-0552/2020, MT834662.1, 8
>MT834661.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-0551/2020, MT834661.1, 8
>MT834657.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-0547/2020, MT834657.1, 8
Error: [blastdbcmd] Entry not found: MW555909.1, MW555909.1, 8
>MT834194.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/hum

Error: [blastdbcmd] Entry not found: MW565741.1, MW565741.1, 6
>MW030220.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/PER/covper084/2020, MW030220.1, 6
>MW030236.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/PER/covper005/2020, MW030236.1, 6
Error: [blastdbcmd] Entry not found: MW565734.1, MW565734.1, 6
>MW276469.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CZB-10590/2020, MW276469.1, 6
Error: [blastdbcmd] Entry not found: MW565728.1, MW565728.1, 6
Error: [blastdbcmd] Entry not found: MW641687.1, MW641687.1, 6
Error: [blastdbcmd] Entry not found: MW643881.1, MW643881.1, 6
>MW030254.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/PER/covper023/2020, MW030254.1, 6
Error: [blastdbcmd] Entry not found: MW565452.1, MW565452.1, 6
>MW031071.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WI-UW-1504/2020, MW031071.1, 6
>MW031073.1 Severe acut

...

>MW223133.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/TX-DSHS-0884/2020 ORF1ab polyprotein (ORF1ab), MW223133.1, 2
Error: [blastdbcmd] Entry not found: MW549246.1, MW549246.1, 2
Error: [blastdbcmd] Entry not found: MW549242.1, MW549242.1, 2
>MW228191.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/TX-DSHS-1055/2020 ORF1ab polyprotein (ORF1ab), MW228191.1, 2
Error: [blastdbcmd] Entry not found: MW549209.1, MW549209.1, 2
>MW228216.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/TX-DSHS-1041/2020 ORF1ab polyprotein (ORF1ab), MW228216.1, 2
Error: [blastdbcmd] Entry not found: MW549221.1, MW549221.1, 2
Error: [blastdbcmd] Entry not found: MW642489.1, MW642489.1, 2
Error: [blastdbcmd] Entry not found: MW637851.1, MW637851.1, 2
>MW228201.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/TX-DSHS-1050/2020 ORF1ab polyprotein (ORF1ab), MW228201.1, 2
>MW228192.1 Severe acu

>MT241668.1 Leopoldamys sabanus voucher MZF1958 mitochondrion, MT241668.1, 1618
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 434
Error: [blastdbcmd] Entry not found: CNA0013710, CNA0013710, 272
>X51700.1 Bos taurus mRNA for bone Gla protein, 7, 140
>AC123074.2 Mus musculus BAC clone RP23-168H19 from 10, AC123074.2, 42
>MK280367.1 Homo sapiens lncAB371.6 lncRNA gene, MK280367.1, 34
>MK279923.1 Homo sapiens lncAB599.3 lncRNA gene, MK279923.1, 26
Error: [blastdbcmd] Entry not found: 2, 2, 14
>AC105600.5 Rattus norvegicus 4 BAC CH230-209E1 (Children's Hospital Oakland Research Institute) complete sequence, AC105600.5, 14
>MK280359.1 Homo sapiens lncAB370.3 lncRNA gene, MK280359.1, 8
>MW332979.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WI-UW-1863/2020, MW332979.1, 2
>MW134275.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CZB-3871/2020 ORF1ab polyprotein (ORF1ab) and ORF1a polyprotein (ORF1ab) ge

>X51700.1 Bos taurus mRNA for bone Gla protein, 7, 9702
Error: [blastdbcmd] Entry not found: CNA0013710, CNA0013710, 4040
>AC123074.2 Mus musculus BAC clone RP23-168H19 from 10, AC123074.2, 3806
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 1594
>MT241668.1 Leopoldamys sabanus voucher MZF1958 mitochondrion, MT241668.1, 994
>AC105600.5 Rattus norvegicus 4 BAC CH230-209E1 (Children's Hospital Oakland Research Institute) complete sequence, AC105600.5, 788
Error: [blastdbcmd] Entry not found: 2, 2, 638
>MT451205.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/AUS/VIC313/2020 ORF1ab polyprotein (ORF1ab), MT451205.1, 320
Error: [blastdbcmd] Entry not found: MW638235.1, MW638235.1, 280
>MW490886.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-2293/2020, MW490886.1, 280
>MW056121.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-1496/2020 ORF1ab polyprotein (ORF1ab), M

>MT952701.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/VA-DCLS-1047/2020, MT952701.1, 2
>MT953980.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/IND/GBRC-383a/2020, MT953980.1, 2
Error: [blastdbcmd] Entry not found: MW638593.1, MW638593.1, 2
>MT953991.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/IND/GBRC-384b/2020, MT953991.1, 2
>MT954922.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/IND/GBRC-380b/2020 ORF1ab polyprotein (ORF1ab), MT954922.1, 2
>MT955114.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/IND/GBRC-395a/2020, MT955114.1, 2
>MT955326.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/IND/GBRC-395b/2020, MT955326.1, 2
Error: [blastdbcmd] Entry not found: MW638586.1, MW638586.1, 2
>MT956771.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/FL-BPHL-1161/2020 ORF1ab polyprotein (OR

SRA: CRR125938
SRA: CRR125939
SRA: CRR125940


In [30]:
for ke, va in sra_results.items():
    print(f'{ke}:')
    for t,v,c in zip(va[0], va[1], va[2]):
        if not t.startswith('Error') and not 'Severe acute respiratory syndrome coronavirus 2' in t:
            print(f'{t}, {v}, {c}')
    print(f'\n')
    

CRR125934:
>MF164268.1 Homo sapiens clone BAC JH12 genomic sequence, MF164268.1, 5816
>MT241668.1 Leopoldamys sabanus voucher MZF1958 mitochondrion, MT241668.1, 4334
>X51700.1 Bos taurus mRNA for bone Gla protein, 7, 3028
>MK280367.1 Homo sapiens lncAB371.6 lncRNA gene, MK280367.1, 890
>MK279923.1 Homo sapiens lncAB599.3 lncRNA gene, MK279923.1, 814
>AJ289709.1 Human endogenous retrovirus H HERV-H/env62 proviral copy, AJ289709.1, 652
>AC123074.2 Mus musculus BAC clone RP23-168H19 from 10, AC123074.2, 556
>AC105600.5 Rattus norvegicus 4 BAC CH230-209E1 (Children's Hospital Oakland Research Institute) complete sequence, AC105600.5, 132
>X68321.1 B.taurus mRNA for cyclin A, 9, 96
>U13856.1 pGEX-5X1 cloning vector, U13856.1, 76
>KY926444.1 Cloning vector pET-hsp12, KY926444.1, 60
>MN996532.2 Bat coronavirus RaTG13, MN996532.2, 56
>MK280359.1 Homo sapiens lncAB370.3 lncRNA gene, MK280359.1, 46
>MN996867.1 Cloning vector pcDNA3.1_+, MN996867.1, 10
>X17276.1 Giant Panda satellite 1 DNA, 4, 8


In [11]:
#flu_titles = [i for i in titles if 'influenza' in i.lower()]
#flu_titles